In [41]:
import os
import json
import re

import pandas as pd
import numpy as np

# Json + CSV 데이터 병합

In [237]:
# 상위 폴더로부터 하위 디렉토리 리스트로 뽑아주는 함수
def load_directory_data(pwd):

    file_path_list = []
    
    # 디렉토리, 디렉토리 내 폴더 리스트, 파일 리스트
    for path,dirs,files in os.walk(pwd):
        
        for f in files:           
            file_path = path + '/' +f
            file_path_list.append(file_path) 
            
    print(len(file_path_list))
    return file_path_list

FILE_1 = load_directory_data("_JSON 1 Instagram JSON File")
print(FILE_1[:5])
print()
FILE_2 = load_directory_data("_JSON 2 Image OCR Result CSV File")
print(FILE_2[:5])

74
['_JSON 1 Instagram JSON File/1day1poem.json', '_JSON 1 Instagram JSON File/9_ruumy.json', '_JSON 1 Instagram JSON File/ajaegeul.json', '_JSON 1 Instagram JSON File/book_jjung.json', '_JSON 1 Instagram JSON File/by_yell7.json']

70
['_JSON 2 Image OCR Result CSV File/1day1poem.csv', '_JSON 2 Image OCR Result CSV File/9_ruumy.csv', '_JSON 2 Image OCR Result CSV File/ajaegeul.csv', '_JSON 2 Image OCR Result CSV File/book_jjung.csv', '_JSON 2 Image OCR Result CSV File/by_yell7.csv']


In [238]:
def read_json(FILE_PATH):
    
    # read json
    with open(FILE_PATH, encoding='UTF8') as json_file:
        json_data = json.load(json_file)
        
    user_name = os.path.basename(FILE_PATH)[:-5]
    insta_df = pd.DataFrame()

    for d in json_data:
        hashtag_list = []
        caption_txt = ''
        insta = pd.DataFrame()

        # caption 해시태그 가져오기
        if 'caption' in d:
            # 간단한 전처리
            caption_txt = d['caption'].replace('\n',' ')
            if caption_txt[0] == '-':
                caption_txt = caption_txt[1:]
            caption_txt = caption_txt.strip()
            hashtag_list = re.findall(r"#(\w+)", caption_txt) 

        # 기존 코드는 caption이 없을 경우 작동 못함
        # comments에서 해시태그 가져오기 
        if 'comments' in d:
            comments = d['comments']
            for c in list(d['comments']):
                # 댓글 작성자가 인스타 유저라면 코멘트 가져오기
                if c['author'] == userName:
                    comment_txt = c['comment']
                    hashtag_list = re.findall(r"#(\w+)", comment_txt)   
                    #태그가 저장이 되었다면 for 문 종료
                    if len(hashtag_list) != 0:
                        break
        
        # 매 loop마다 데이터 쌓기
        insta = pd.DataFrame({'USER_ID': user_name,
                              'CONTENT_ID': d['key'].split('/')[-2],
                              'Content_txt': caption_txt,
                              'Hashtags' : [hashtag_list]})
        insta_df = insta_df.append(insta)
        insta_df = insta_df.reset_index(drop=True)

    return insta_df

In [239]:
def read_csv(FILE_PATH):
    data = pd.read_csv(FILE_PATH,encoding = 'utf-8' )
    data = data.iloc[:,:2]
    data.columns = ['CONTENT_IMAGE_ID','Image_Content_txt']
    # 게시글 아이디 
    data['CONTENT_IMAGE_ID'] = data['CONTENT_IMAGE_ID'].apply(lambda x : x[:-4])
    # 게시글에 여러장 있을 경우 추가 구분
    data['CONTENT_ID'] =  data['CONTENT_IMAGE_ID'].apply(lambda x : x[:-2])
    data['USER_ID'] = os.path.basename(FILE_PATH)[:-4]
    data = data[['USER_ID','CONTENT_ID','CONTENT_IMAGE_ID','Image_Content_txt']]
    
    return data

In [240]:
def merging(FILE_LIST1,FILE_LIST2):
    total_data = 0
    for f_json in FILE_LIST1:
        user_json = os.path.basename(f_json)[:-5]
        insta_data = read_json(f_json)
        for f_csv in FILE_LIST2:
            user_csv = os.path.basename(f_csv)[:-4]
            # 유저 같다면
            if user_json == user_csv:
                ocr_data = read_csv(f_csv)
                print(user_csv)
                merge_df = pd.merge(insta_data,ocr_data,on=['USER_ID','CONTENT_ID'],how='inner')
                merge_df = merge_df[['USER_ID','CONTENT_ID','CONTENT_IMAGE_ID','Image_Content_txt','Content_txt','Hashtags']]
                merge_df.to_csv('Final_Data/' + user_json + '.csv', encoding='utf-8-sig')
                total_data += len(merge_df)
    print("총 데이터 개수",total_data)
merging(FILE_1,FILE_2)                

1day1poem
9_ruumy
ajaegeul
book_jjung
by_yell7
chaemss
choejemin_pvc
choi_dol
churihyung
c___w00
dajeong_geul
day_in_scene
dden_by
dearbliss2
deep_bak
dh_solace
doodler_1211
funnyprince81
galin001
geulgomm
guitarlist._w.g
hanl_i
haru11__
haru_line
h_01_00_a
h_jongduck
iamkimbunny
insightsh
insta_book_cafe
insum_
iris_daily_writing
jaulounge
jea.therapy
jinagann
jinsimgeul
jms14219
kiheaven97
kim_friendship_01.19
kim_hanwoong
laenari
marom__story
me______new
moonfatalae
namumind
ok_yeong_
poem_1000_
reason_that_i_live
riosniper114
seesaw517
see_min0727
seodeokjun
seokgeul
shine_like_september
syhiphop
taeeeseok
taehee_editor
tale_tree_
violet_hoho
w.gahee
w.ojoo
wan_e0612
woojin_940205
worker_poet
writer.jieun
writing_peach.97
youandme.211
yumradio
_illusion_30
_neobam
총 데이터 개수 31613


# 전체 데이터로 합치기

In [253]:
FILE_Final = load_directory_data("Final_Data")
total_df = pd.DataFrame()

for f in FILE_Final:
    df =pd.read_csv(f,encoding = 'utf-8')
    df = df.iloc[:,1:]
    total_df = total_df.append(df,sort=True)
    
total_df = total_df.iloc[:,:-1]
total_df = total_df[['USER_ID','CONTENT_ID', 'CONTENT_IMAGE_ID','Image_Content_txt','Content_txt', 'Hashtags']]
total_df = total_df.reset_index(drop = True)
total_df.head()

71


,USER_ID,CONTENT_ID,CONTENT_IMAGE_ID,Image_Content_txt,Content_txt,Hashtags
0,1day1poem,B3B7tNPFOhY,B3B7tNPFOhY_0,벌레 물어 가렵다면 할머니 침을 발라주셨다. 쪼그려 앉아 다리 저릴 때도 할머니 코...,벌레 물어 가렵다면 할머니 침을 발라주셨다. 쪼그려 앉아 다리 저릴 때도 할머니 ...,"['가능하면1일1시', '190825', '임재건', '1일1시', '다시는사랑이없..."
1,1day1poem,B3A90BZlUGb,B3A90BZlUGb_0,들에 억새가 은발에 구부정하니 할매 닮아서 할매랑 들에 가면 할매 못 찾겠네. -할...,들에 억새가 은발에 구부정하니 할매 닮아서 할매랑 들에 가면 할매 못 찾겠네. -...,"['가능하면1일1시', '190930', '임재건', '1일1시', '다시는사랑이없..."
2,1day1poem,B2_T0kQlsWv,B2_T0kQlsWv_0,가만히 하늘을 살펴보면 하늘서 구름이 기장 큰 체를 한다. 사람 속살필 적 걱정이 ...,가만히 하늘을 살펴보면 하늘서 구름이 가장 큰 체를 한다. 사람 속 살필 적 걱정...,"['가능하면1일1시', '190824', '임재건', '1일1시', '다시는사랑이없..."
3,1day1poem,B2-WQRrFlNi,B2-WQRrFlNi_0,날줄 몰라도 나무 위 집은 얼마든지을수 있지. -거미집4 # 19.09.29 #가능...,날 줄 몰라도 나무 위 집은 얼마든 지을 수 있지. - 거미집4 오른다면 #가능...,"['가능하면1일1시', '190929', '임재건', '1일1시', '다시는사랑이없..."
4,1day1poem,B28u5bmlGaV,B28u5bmlGaV_0,달력은 입추부터 가을이라하고 TV는 이번 비가그치면 가을이라 하네요. 어머니는 초록...,달력은 입추부터 가을이라 하고 TV는 이번 비가 그치면 가을이라 하네요. 어머니...,"['가능하면1일1시', '190823', '임재건', '1일1시', '다시는사랑이없..."
